In [1]:
# Check if GMST signal is projecting onto stdevs.

In [2]:
import numpy as np
import xarray as xr
from distributed import Client
import dask_jobqueue
#import dask
#import daskdataframe as dd
import matplotlib.pyplot as plt
import glob
import netCDF4 as nc
import zarr
import nc_time_axis
import xskillscore as xs
import cartopy as cart
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from scipy.special import erfinv, erf
#%run data_handler.ipynb
#np.set_printoptions(threshold=np.inf)

In [3]:
# #Chicago
# LAT        = 41.8781
# LON        = 360-87.6298
# # #Bengaluru:
LAT        = 12.9716
LON        = 77.5946
########## CONUS ############
#CONUS lat-lon
top     = 50.0 # north lat
left    = -124.7844079+360 # west long
right   = -66.9513812+360 # east long
bottom  =  24.7433195 # south lat
################################
cesm2_path = '/global/scratch/users/harsha/LENS/cesm2/tasmax/'
cvals      = '/global/scratch/users/harsha/LENS/cesm2/cvals/'
loc        = 'Bengaluru/'
locn       = 'Bengaluru'
cvals1     = '/global/scratch/users/harsha/LENS/cesm2/tmax_mem'
## Change/Check these parameters before running
# season     = 'JJA'
# season0    = 'jja'
# months     = [6,7,8]
# day        =  2111      #First day of the season
# Day        = 'Jul30'

In [4]:
def mean_adj(ds_stack,ds_mroll):
    ds = (ds_stack - ds_stack.mean(dim='mtime')) + ds_mroll
    return ds

def mean_std_adj(ds_stack,ds_mroll,ds_sroll):
    ds = (ds_stack -ds_stack.mean(dim='mtime'))*(ds_sroll/ds_stack.std(dim='mtime'))\
          + ds_mroll
    return ds

In [5]:
def gmst(arr, dims):
        #Compute GMST #g = global, a= annual, dims could be days, members etc
        weights         = np.cos(np.deg2rad(arr.lat))
        weights.name    = "weights"
        arr_weighted    = arr.weighted(weights)
        arr_gmean       = arr_weighted.mean({'lat','lon'},skipna=True)
        ####
        arr_gmst        = arr_gmean.mean(dims)       
        return arr_gmst

In [6]:
def to_daily(ds):
    year = ds.time.dt.year
    day = ds.time.dt.dayofyear

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), day=("time", day.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(time=("year", "day")).unstack("time")  

In [7]:
def stacker(x):
    #Assumes that x has dimensions year and member
    x = x.stack(ym=('year','member'),create_index=False)
    return x
def stacker2(x):
    #Assumes that x has dimensions year and member
    x = x.stack(mtime=('year','member'),create_index=False)
    return x

In [8]:
def stack_roll(ds,roll_len,stride_len):
    #Assumes that ds has dimensions year and member
    ds_stack   = stacker(ds)
    #print(ds_stack)
    ds_roll    = ds_stack.rolling(ym=roll_len, center=True).construct('index', stride=stride_len)
    
    return ds_roll   

In [9]:
def global_annual_rmse(arr_err, dim):
      #Compute spatially weighted global mean of square error
      # and then compute annual rmse #g = global, a= annual
        arr_err_sq            = arr_err**2
        weights               = np.cos(np.deg2rad(arr_err_sq.lat))
        weights.name          = "weights"
        arr_serror_weighted   = arr_err_sq.weighted(weights)
        arr_serror_gmean      = arr_serror_weighted.mean({'lat','lon'},skipna=True)
        ####
        arr_gamse             = arr_serror_gmean.mean(dim)
        arr_garmse            = np.sqrt(arr_gamse)
        
        return arr_garmse

In [10]:
job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
#job_extra =['--qos=lr6_lowprio','--account=ac_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=lr_lowprio','--account=ac_cumulus']
cluster = dask_jobqueue.SLURMCluster(queue="cf1", cores=10, walltime='6:00:00', 
                local_directory='/global/scratch/users/harsha/dask_space/', 
                log_directory='/global/scratch/users/harsha/dask_space/', 
                job_extra_directives=job_extra, interface='eth0', memory="192GB") 
client  = Client(cluster) 
cluster.scale_up(5)

In [11]:
cluster

In [12]:
pi_year0 = '1850'
pi_year1 = '1879'
ic_year0 = '2071'
ic_year1 = '2100'
#doy      = 211 # day_of_year
#date     = 'Jul30'

In [13]:
%%time
ds_ssp  = xr.open_mfdataset(cesm2_path+'ssp370_cesm2tmax.zarr',engine='zarr')
ds_hist = xr.open_mfdataset(cesm2_path+'bhist_cesm2tmax.zarr',engine='zarr')

CPU times: user 569 ms, sys: 91.3 ms, total: 660 ms
Wall time: 744 ms


In [14]:
%%time
#t-temp, nw - no-warming, w-warming, hw = historical warming, g=global
tgw_eoc = ds_ssp.TREFHTMX.sel(time=slice(ic_year0,ic_year1))
         #.sel(lat=slice(bottom,top))
         #.sel(lon=slice(left,right))
tgnw   = ds_hist.TREFHTMX.sel(time=slice(pi_year0,pi_year1))
         #.sel(lon=slice(left,right)).sel(lat=slice(bottom,top))
tghw   = ds_hist.TREFHTMX
         #.sel(lat=slice(bottom,top))
         #.sel(lon=slice(left,right))
tgnw

CPU times: user 139 ms, sys: 1.88 ms, total: 141 ms
Wall time: 138 ms


,Array,Chunk
Bytes,225.56 GiB,769.92 MiB
Shape,"(10950, 192, 288, 100)","(3650, 192, 288, 1)"
Count,2001 Tasks,300 Chunks
Type,float32,numpy.ndarray


In [15]:
######### Seasons ########
tgw_pi_djf = tgnw.sel(time=tgnw.time.dt.month.isin([12,1,2]))
tgw_pi_mam = tgnw.sel(time=tgnw.time.dt.month.isin([3,4,5]))
tgw_pi_jja = tgnw.sel(time=tgnw.time.dt.month.isin([6,7,8]))
tgw_pi_son = tgnw.sel(time=tgnw.time.dt.month.isin([9,10,11]))
#
tgw_eoc_djf = tgw_eoc.sel(time=tgw_eoc.time.dt.month.isin([12,1,2]))
tgw_eoc_mam = tgw_eoc.sel(time=tgw_eoc.time.dt.month.isin([3,4,5]))
tgw_eoc_jja = tgw_eoc.sel(time=tgw_eoc.time.dt.month.isin([6,7,8]))
tgw_eoc_son = tgw_eoc.sel(time=tgw_eoc.time.dt.month.isin([9,10,11]))
tgw_eoc_son

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


,Array,Chunk
Bytes,56.24 GiB,191.95 MiB
Shape,"(2730, 192, 288, 100)","(910, 192, 288, 1)"
Count,1701 Tasks,400 Chunks
Type,float32,numpy.ndarray


In [16]:
tgw_pid_djf  = to_daily(tgw_pi_djf)
tgw_pid_mam  = to_daily(tgw_pi_mam)
tgw_pid_jja  = to_daily(tgw_pi_jja)
tgw_pid_son  = to_daily(tgw_pi_son)
#
tgw_eocd_djf = to_daily(tgw_eoc_djf)
tgw_eocd_mam = to_daily(tgw_eoc_mam)
tgw_eocd_jja = to_daily(tgw_eoc_jja)
tgw_eocd_son = to_daily(tgw_eoc_son)
tgw_eocd_son

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._unstack_full_reindex(
/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

,Array,Chunk
Bytes,56.24 GiB,191.95 MiB
Shape,"(192, 288, 100, 30, 91)","(192, 288, 1, 10, 91)"
Count,2501 Tasks,400 Chunks
Type,float32,numpy.ndarray


In [17]:
# eoc_gmst  = gmst(tgw_eocd,{'day','member'})
# pi_gmst   = gmst(tgw_pid,{'day','member'})
# eoc_gmst
#
#eoc_gmsta.values
#
# eoc_gmst_change = eoc_gmst.sel(year= 2100) - eoc_gmst.sel(year=2071)
# eoc_gmst_change.values

In [18]:
%%time
# eoc_gmst.plot()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


In [19]:
#with dask.config.set(**{'array.slicing.split_large_chunks': False}):

In [20]:
%%time
# pi_gmst.plot()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [21]:
########## Detrending daily tasmax series ###############

In [22]:
%%time
#DJF
pcoeffs_eoc_djf = tgw_eocd_djf.mean('member').polyfit(dim='year',deg=1)
pcoeffs_pi_djf  = tgw_pid_djf.mean('member').polyfit(dim='year',deg=1)
#MAM
pcoeffs_eoc_mam = tgw_eocd_mam.mean('member').polyfit(dim='year',deg=1)
pcoeffs_pi_mam  = tgw_pid_mam.mean('member').polyfit(dim='year',deg=1)
#JJA
pcoeffs_eoc_jja = tgw_eocd_jja.mean('member').polyfit(dim='year',deg=1)
pcoeffs_pi_jja  = tgw_pid_jja.mean('member').polyfit(dim='year',deg=1)
#SON
pcoeffs_eoc_son = tgw_eocd_son.mean('member').polyfit(dim='year',deg=1)
pcoeffs_pi_son  = tgw_pid_son.mean('member').polyfit(dim='year',deg=1)
# pcoeffs_eoc = pcoeffs_eoc.chunk({'lat':10,'lon':12})
# pcoeffs_pi  = pcoeffs_pi.chunk({'lat':10,'lon':12})
pcoeffs_eoc_son

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/variable.py:1722: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._stack_once(dims, new_dim)
/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/variable.py:1722: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(

CPU times: user 3.01 s, sys: 3.62 s, total: 6.63 s
Wall time: 6.35 s


<xarray.Dataset>
Dimensions:               (lat: 192, lon: 288, day: 91, degree: 2)
Coordinates:
  * lat                   (lat) float64 -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon                   (lon) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
  * day                   (day) int64 244 245 246 247 248 ... 331 332 333 334
  * degree                (degree) int64 1 0
Data variables:
    polyfit_coefficients  (degree, lat, lon, day) float64 dask.array<chunksize=(2, 192, 288, 91), meta=np.ndarray>

In [38]:
# Use polyfit to get the slope (m) and intercept (c)
m_eoc_djf     = pcoeffs_eoc_djf.polyfit_coefficients.sel(degree=1)
c_eoc_djf     = pcoeffs_eoc_djf.polyfit_coefficients.sel(degree=0)
m_eoc_mam     = pcoeffs_eoc_mam.polyfit_coefficients.sel(degree=1)
c_eoc_mam     = pcoeffs_eoc_mam.polyfit_coefficients.sel(degree=0)
m_eoc_jja     = pcoeffs_eoc_jja.polyfit_coefficients.sel(degree=1)
c_eoc_jja     = pcoeffs_eoc_jja.polyfit_coefficients.sel(degree=0)
m_eoc_son     = pcoeffs_eoc_son.polyfit_coefficients.sel(degree=1)
c_eoc_son     = pcoeffs_eoc_son.polyfit_coefficients.sel(degree=0)

# Calculate the corrected intercept (c') to make sure the y-intercept at 2085 is zero
k_eoc_djf = -m_eoc_djf * 2085 - c_eoc_djf
k_eoc_mam = -m_eoc_mam * 2085 - c_eoc_mam
k_eoc_jja = -m_eoc_jja * 2085 - c_eoc_jja
k_eoc_son = -m_eoc_son * 2085 - c_eoc_son

# Compute the trend using m and c_prime
trend_eoc_djf = m_eoc_djf * tgw_eocd_djf['year'] + k_eoc_djf
trend_eoc_mam = m_eoc_mam * tgw_eocd_mam['year'] + k_eoc_mam
trend_eoc_jja = m_eoc_jja * tgw_eocd_jja['year'] + k_eoc_jja
trend_eoc_son = m_eoc_son * tgw_eocd_son['year'] + k_eoc_son
# Detrend the data
det_tgw_eoc_djf = tgw_eocd_djf - trend_eoc_djf
det_tgw_eoc_mam = tgw_eocd_mam - trend_eoc_mam
det_tgw_eoc_jja = tgw_eocd_jja - trend_eoc_jja
det_tgw_eoc_son = tgw_eocd_son - trend_eoc_son
#Chunk
det_tgw_eoc_djf = det_tgw_eoc_djf.chunk({'lat':40,'lon':50,'year':10})
det_tgw_eoc_mam = det_tgw_eoc_mam.chunk({'lat':40,'lon':50,'year':10})
det_tgw_eoc_jja = det_tgw_eoc_jja.chunk({'lat':40,'lon':50,'day':5})
det_tgw_eoc_son = det_tgw_eoc_son.chunk({'lat':40,'lon':50,'day':5})
det_tgw_eoc_djf

,Array,Chunk
Bytes,111.24 GiB,13.73 MiB
Shape,"(192, 288, 100, 30, 90)","(40, 50, 1, 10, 90)"
Count,30482 Tasks,9000 Chunks
Type,float64,numpy.ndarray


In [24]:
%%time
# Use polyfit to get the slope (m) and intercept (c)
m_pi_djf    = pcoeffs_pi_djf.polyfit_coefficients.sel(degree=1)
c_pi_djf    = pcoeffs_pi_djf.polyfit_coefficients.sel(degree=0)
m_pi_mam    = pcoeffs_pi_mam.polyfit_coefficients.sel(degree=1)
c_pi_mam    = pcoeffs_pi_mam.polyfit_coefficients.sel(degree=0)
m_pi_jja    = pcoeffs_pi_jja.polyfit_coefficients.sel(degree=1)
c_pi_jja    = pcoeffs_pi_jja.polyfit_coefficients.sel(degree=0)
m_pi_son    = pcoeffs_pi_son.polyfit_coefficients.sel(degree=1)
c_pi_son    = pcoeffs_pi_son.polyfit_coefficients.sel(degree=0)

# Calculate the corrected intercept (c') to make sure the y-intercept at 2085 is zero
k_pi_djf = -m_pi_djf * 1865 - c_pi_djf
k_pi_mam = -m_pi_mam * 1865 - c_pi_mam
k_pi_jja = -m_pi_jja * 1865 - c_pi_jja
k_pi_son = -m_pi_son * 1865 - c_pi_son

# Compute the trend using m and c_prime
trend_pi_djf = m_pi_djf * tgw_pid_djf['year'] + k_pi_djf
trend_pi_mam = m_pi_mam * tgw_pid_mam['year'] + k_pi_mam
trend_pi_jja = m_pi_jja * tgw_pid_jja['year'] + k_pi_jja
trend_pi_son = m_pi_son * tgw_pid_son['year'] + k_pi_son

# Detrend the data
det_tgw_pi_djf = tgw_pid_djf - trend_pi_djf
det_tgw_pi_mam = tgw_pid_mam - trend_pi_mam
det_tgw_pi_jja = tgw_pid_jja - trend_pi_jja
det_tgw_pi_son = tgw_pid_son - trend_pi_son
#Chunk
det_tgw_pi_djf = det_tgw_pi_djf.chunk({'lat':15,'lon':20,'day':5})
det_tgw_pi_mam = det_tgw_pi_mam.chunk({'lat':15,'lon':20,'day':5})
det_tgw_pi_jja = det_tgw_pi_jja.chunk({'lat':15,'lon':20,'day':5})
det_tgw_pi_son = det_tgw_pi_son.chunk({'lat':15,'lon':20,'day':5})
#
det_tgw_pi_djf = det_tgw_pi_djf.rename('pi_djf_det')
det_tgw_eoc_djf = det_tgw_eoc_djf.rename('eoc_djf_det')

CPU times: user 1min 15s, sys: 2.24 s, total: 1min 17s
Wall time: 1min 17s


In [25]:
#### season ######
detpath             = cvals+'detrended/'
tgw_djf_pidetpath   = detpath+'tgw_djf_pidet'+pi_year0+'_'+pi_year1+'.zarr'
tgw_djf_eocdetpath  = detpath+'tgw_djf_eocdet'+ic_year0+'_'+ic_year1+'.zarr'
# qgw_mam_pidetpath   = detpath+'qgw_mam_pidet'+pi_year0+'_'+pi_year1+'.zarr'
# qgw_mam_eocdetpath  = detpath+'qgw_mam_eocdet'+ic_year0+'_'+ic_year1+'.zarr'
# qgw_jja_pidetpath   = detpath+'qgw_jja_pidet'+pi_year0+'_'+pi_year1+'.zarr'
# qgw_jja_eocdetpath  = detpath+'qgw_jja_eocdet'+ic_year0+'_'+ic_year1+'.zarr'
# qgw_son_pidetpath   = detpath+'qgw_son_pidet'+pi_year0+'_'+pi_year1+'.zarr'
# qgw_son_eocdetpath  = detpath+'qgw_son_eocdet'+ic_year0+'_'+ic_year1+'.zarr'

In [36]:
# %%time
# det_tgw_pi_djf.to_dataset().to_zarr(tgw_djf_pidetpath)

In [ ]:
%%time
det_tgw_pi_djf  = det_tgw_pi_djf.rename('pi_djf_det')
det_tgw_eoc_djf = det_tgw_eoc_djf.rename('eoc_djf_det')
det_tgw_eoc_djf.to_dataset().to_zarr(tgw_djf_eocdetpath,mode='w')

In [28]:
%%time
#Stack member and year into one dimension- 'ym'. Then, roll over this dimension with stride = N_mem.
#This works because of the way xarray stacks two dimensions. window_len =30. 
#I have called the 'window' dimension 'index'.
quants             = np.arange(0,1.01,0.01)
########################
N_mem              = 100
window_len         = 30
###### season #######
tgw_djf_pidetroll    = stack_roll(det_tgw_pi_djf,N_mem*window_len,N_mem)
tgw_djf_eocdetroll   = stack_roll(det_tgw_eoc_djf,N_mem*window_len,N_mem)
tgw_mam_pidetroll    = stack_roll(det_tgw_pi_mam,N_mem*window_len,N_mem)
tgw_mam_eocdetroll   = stack_roll(det_tgw_eoc_mam,N_mem*window_len,N_mem)
tgw_jja_pidetroll    = stack_roll(det_tgw_pi_jja,N_mem*window_len,N_mem)
tgw_jja_eocdetroll   = stack_roll(det_tgw_eoc_jja,N_mem*window_len,N_mem)
tgw_son_pidetroll    = stack_roll(det_tgw_pi_son,N_mem*window_len,N_mem)
tgw_son_eocdetroll   = stack_roll(det_tgw_eoc_son,N_mem*window_len,N_mem)
#########
tgw_djf_pidetroll0   = tgw_djf_pidetroll.sel(ym=[15])  
tgw_djf_eocdetroll0  = tgw_djf_eocdetroll.sel(ym=[15])  
tgw_mam_pidetroll0   = tgw_mam_pidetroll.sel(ym=[15])  
tgw_mam_eocdetroll0  = tgw_mam_eocdetroll.sel(ym=[15])  
tgw_jja_pidetroll0   = tgw_jja_pidetroll.sel(ym=[15])  
tgw_jja_eocdetroll0  = tgw_jja_eocdetroll.sel(ym=[15])  
tgw_son_pidetroll0   = tgw_son_pidetroll.sel(ym=[15])  
tgw_son_eocdetroll0  = tgw_son_eocdetroll.sel(ym=[15])  
tgw_son_pidetroll

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/variable.py:1722: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._stack_once(dims, new_dim)
/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/variable.py:1722: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(

CPU times: user 41.3 s, sys: 993 ms, total: 42.3 s
Wall time: 42 s


,Array,Chunk
Bytes,3.30 TiB,1.01 GiB
Shape,"(192, 288, 91, 30, 3000)","(15, 20, 5, 30, 3000)"
Count,3911003 Tasks,3705 Chunks
Type,float64,numpy.ndarray


In [29]:
%%time
#Compute quantiles 
qgw_djf_pidet    = tgw_djf_pidetroll0.quantile(quants,dim='index',skipna=False)
qgw_djf_eocdet   = tgw_djf_eocdetroll0.quantile(quants,dim='index',skipna=False)
qgw_mam_pidet    = tgw_mam_pidetroll0.quantile(quants,dim='index',skipna=False)
qgw_mam_eocdet   = tgw_mam_eocdetroll0.quantile(quants,dim='index',skipna=False)
qgw_jja_pidet    = tgw_jja_pidetroll0.quantile(quants,dim='index',skipna=False)
qgw_jja_eocdet   = tgw_jja_eocdetroll0.quantile(quants,dim='index',skipna=False)
qgw_son_pidet    = tgw_son_pidetroll0.quantile(quants,dim='index',skipna=False)
qgw_son_eocdet   = tgw_son_eocdetroll0.quantile(quants,dim='index',skipna=False)
#Name the array
qgw_djf_pidet    = qgw_djf_pidet.rename('pi_djf_qdet')
qgw_djf_eocdet   = qgw_djf_pidet.rename('eoc_djf_qdet')
qgw_mam_pidet    = qgw_mam_pidet.rename('pi_mam_qdet')
qgw_mam_eocdet   = qgw_mam_pidet.rename('eoc_mam_qdet')
qgw_jja_pidet    = qgw_jja_pidet.rename('pi_jja_qdet')
qgw_jja_eocdet   = qgw_jja_pidet.rename('eoc_jja_qdet')
qgw_son_pidet    = qgw_son_pidet.rename('pi_son_qdet')
qgw_son_eocdet   = qgw_son_pidet.rename('eoc_son_qdet')
#
qgw_djf_pidet

CPU times: user 78.4 ms, sys: 79 µs, total: 78.5 ms
Wall time: 75.9 ms


,Array,Chunk
Bytes,3.74 GiB,1.16 MiB
Shape,"(101, 192, 288, 90, 1)","(101, 15, 20, 5, 1)"
Count,3722735 Tasks,3510 Chunks
Type,float64,numpy.ndarray


In [30]:
#### season ######
detpath             = cvals+'detrended/'
qgw_djf_pidetpath   = detpath+'qgw_djf_pidet'+pi_year0+'_'+pi_year1+'.zarr'
qgw_djf_eocdetpath  = detpath+'qgw_djf_eocdet'+ic_year0+'_'+ic_year1+'.zarr'
qgw_mam_pidetpath   = detpath+'qgw_mam_pidet'+pi_year0+'_'+pi_year1+'.zarr'
qgw_mam_eocdetpath  = detpath+'qgw_mam_eocdet'+ic_year0+'_'+ic_year1+'.zarr'
qgw_jja_pidetpath   = detpath+'qgw_jja_pidet'+pi_year0+'_'+pi_year1+'.zarr'
qgw_jja_eocdetpath  = detpath+'qgw_jja_eocdet'+ic_year0+'_'+ic_year1+'.zarr'
qgw_son_pidetpath   = detpath+'qgw_son_pidet'+pi_year0+'_'+pi_year1+'.zarr'
qgw_son_eocdetpath  = detpath+'qgw_son_eocdet'+ic_year0+'_'+ic_year1+'.zarr'
print(qgw_djf_pidetpath)

/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/qgw_djf_pidet1850_1879.zarr


In [31]:
%%time
#### Save files #######
qgw_djf_pidet.to_dataset().to_zarr(qgw_djf_pidetpath,mode='w')

Exception ignored in: <bound method GCDiagnosis._gc_callback of <distributed.utils_perf.GCDiagnosis object at 0x2b7f88a2f7f0>>
Traceback (most recent call last):
  File "/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/distributed/utils_perf.py", line 179, in _gc_callback
    def _gc_callback(self, phase, info):
KeyboardInterrupt: 

KeyboardInterrupt



In [32]:
%%time
qgw_mam_pidet.to_dataset().to_zarr(qgw_mam_pidetpath,mode='w')
qgw_jja_pidet.to_dataset().to_zarr(qgw_jja_pidetpath,mode='w')
qgw_son_pidet.to_dataset().to_zarr(qgw_son_pidetpath,mode='w')


KeyboardInterrupt



In [33]:
%%time
qgw_djf_eocdet.to_dataset().to_zarr(qgw_djf_eocdetpath,mode='w')
qgw_mam_eocdet.to_dataset().to_zarr(qgw_mam_eocdetpath,mode='w')
qgw_jja_eocdet.to_dataset().to_zarr(qgw_jja_eocdetpath,mode='w')
qgw_son_eocdet.to_dataset().to_zarr(qgw_son_eocdetpath,mode='w')


KeyboardInterrupt

